In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import maxjoshua as mh
import tensorflow as tf
import sklearn.preprocessing

## Load dataset

In [2]:
import sklearn.datasets
X, y = sklearn.datasets.make_regression(n_samples=1000, n_features=100, n_informative=20, n_targets=3)

In [3]:
X.shape, y.shape, y.max(), y.min()

((1000, 100), (1000, 3), 792.2004002707565, -988.3130088872593)

## Feature selection

In [4]:
%%time 
indices, values, num_in, num_out = mh.pretrain_submodels(
    sklearn.preprocessing.scale(X), 
    sklearn.preprocessing.scale(y), 
    num_out=64, n_select=3)

0 (0, 63, 84) [0.3242548352417717, -0.07996116590242418, -0.032470841960068636] 0.8953223077048862
1 (64, 80, 86) [0.4231414746682984, 0.23757581425116953, 0.2391301058166192] 0.8469327570104306
2 (6, 16, 86) [0.21547533349695477, 0.037101406868916435, 0.403163651069875] 0.8646054248853232
CPU times: user 16.9 s, sys: 550 ms, total: 17.5 s
Wall time: 16.8 s


## Training

In [5]:
model = tf.keras.models.Sequential([
    mh.SparseLayerAsEnsemble(
        num_in=num_in, num_out=num_out, sp_indices=indices, sp_values=values,
        sp_trainable=False
    ),
    tf.keras.layers.Dense(3, use_bias=True)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=3e-4, beta_1=.9, beta_2=.999, epsilon=1e-7, amsgrad=True),
    loss='mean_squared_error'
)

history = model.fit(X, sklearn.preprocessing.scale(y), epochs=3)
# model.summary()

Epoch 1/3


2022-06-21 00:01:03.267757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


32/32 [==============================] - 1s 3ms/step - loss: 0.9939
Epoch 2/3
32/32 [==============================] - 0s 3ms/step - loss: 0.9190
Epoch 3/3
32/32 [==============================] - 0s 3ms/step - loss: 0.8491


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sparse_layer_as_ensemble (S  (None, 64)               592       
 parseLayerAsEnsemble)                                           
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 787
Trainable params: 395
Non-trainable params: 392
_________________________________________________________________
